# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
import winsound

In [2]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.graph_objs.layout import YAxis,XAxis,Margin
import webbrowser
data['chrome_path'] = "C:/Program Files (x86)/Google/Chrome/Application/chrome.exe %s"

In [3]:
hour_bins = {0:'21-0', 
        1:'1-4', 2:'1-4', 3:'1-4', 4:'1-4', 
        5:'5-8', 6:'5-8', 7:'5-8', 8:'5-8',
        9:'9-12', 10:'9-12', 11:'9-12', 12:'9-12',
        13:'13-16', 14:'13-16', 15:'13-16', 16:'13-16',
        17:'17-20', 18:'17-20', 19:'17-20', 20:'17-20',
        21:'21-0', 22:'21-0', 23:'21-0'}

weekday_dict = {0:'Sunday' , 1:'Monday', 2:'Tuesday', 3:'Wednesday', 4:'Thursday', 5:'Friday', 6:'Saturday'}

In [4]:
%%time
data = get_date_list(data)
data = read_data(data)

Record num : 32763877
Wall time: 1min 28s


In [5]:
%%time
data['df']['tick']      = (data["df"]['Ask'] + data["df"]['Bid'])/2
data['df']['dt']   = [dt.datetime.strptime(x.split(".")[0],"%Y%m%d %H:%M:%S") for x in data["df"]['DateTime']]

data['df'] = data['df'][['dt','tick']]
data['df']['month_val'] = [x.month for x in data['df']['dt']]
data['df']['date_val'] = [x.day for x in data['df']['dt']]
data['df']['week_day'] = [x.weekday() for x in data['df']['dt']]
data['df'] = data['df'].replace({'week_day' : weekday_dict})
data['df'].head(2)

Wall time: 8min 44s


,dt,tick,month_val,date_val,week_day
0,2020-01-01 22:01:12,1.12133,1,1,Tuesday
1,2020-01-01 22:01:17,1.12140,1,1,Tuesday


In [6]:
%%time
idx = data['df'].groupby(['date_val', 'month_val'])['tick'].transform(max) == data['df']['tick']
data['df_max'] = data['df'][idx]
data['df_max']['hour_val'] = [x.hour for x in data['df_max']['dt']]
del data['df_max']['dt']
data['df_max'] = data['df_max'].drop_duplicates()
data['df_max']['hour_bins'] = data['df_max']['hour_val'].map(hour_bins)


idx = data['df'].groupby(['date_val', 'month_val'])['tick'].transform(min) == data['df']['tick']
data['df_min'] = data['df'][idx]
data['df_min']['hour_val'] = [x.hour for x in data['df_min']['dt']]
del data['df_min']['dt']
data['df_min'] = data['df_min'].drop_duplicates()
data['df_min']['hour_bins'] = data['df_min']['hour_val'].map(hour_bins)

Wall time: 3.51 s


In [11]:
vc = data['df_max']['hour_bins'].value_counts()
x = vc.index
y = vc.values

data['df_counts'] = pd.DataFrame()
data['df_counts']['hour_bins'] = x
data['df_counts']['count'] = y
data['df_counts']['type'] = 'max' 


vc = data['df_min']['hour_bins'].value_counts()
x = vc.index
y = vc.values

data['temp'] = pd.DataFrame()
data['temp']['hour_bins'] = x
data['temp']['count'] = y
data['temp']['type'] = 'min' 

data['df_counts'] = data['df_counts'].append(data['temp'])

In [12]:
fig = px.bar(data_frame = data['df_counts'], x = 'hour_bins' , y = 'count' ,text = 'count', color = 'type')

fig.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 1,
        dtick = 1,
    )
)


file_path = (f'{os.getcwd()}\data\Analysis_{data["input_year"]}.html')
fig.write_html(file_path)
webbrowser.get(data['chrome_path']).open(file_path)

True

vc = data['df_max']['week_day'].value_counts()
x = vc.index
y = vc.values

data['df_counts'] = pd.DataFrame()
data['df_counts']['week_bins'] = x
data['df_counts']['count'] = y
data['df_counts']['type'] = 'max' 


vc = data['df_min']['week_day'].value_counts()
x = vc.index
y = vc.values

data['temp'] = pd.DataFrame()
data['temp']['week_bins'] = x
data['temp']['count'] = y
data['temp']['type'] = 'min' 

data['df_counts'] = data['df_counts'].append(data['temp'])

fig = px.bar(data_frame = data['df_counts'], x = 'week_bins' , y = 'count' ,text = 'count', color = 'type')

fig.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 1,
        dtick = 1,
    )
)


file_path = (f'{os.getcwd()}\data\Analysis_{data["input_year"]}.html')
fig.write_html(file_path)
webbrowser.get(data['chrome_path']).open(file_path)